# Portfolio Optimization Project

# 📑 Table of Contents

Welcome to the **Portfolio Optimization Project** — a full walk-through of modern portfolio management concepts using Python, Jupyter, and financial theory. 

---

## Part I — Foundations: Statistics & Efficient Frontiers
- [1. Statistical Analysis & KPIs](#1-statistical-analysis--kpis)  
  Compute returns, volatility, Sharpe/Sortino ratios, skewness, kurtosis, correlations, and cumulative charts.  
- [2. Two-Asset Efficient Frontier](#2-two-asset-efficient-frontier)  
  Visualize diversification benefits with a simple two-asset trade-off.  
- [3. Three-Asset Efficient Frontier](#3-three-asset-efficient-frontier)  
  Extend frontier analysis to three assets using 3D plots.  
- [4. Multi-Asset Efficient Frontier](#4-multi-asset-efficient-frontier)  
  Construct the full frontier with all assets; compare unconstrained vs. constrained versions.  

---

## Part II — Optimization & Allocation
- [5. Mean–Variance Optimization (MVO)](#5-meanvariance-optimization-mvo)  
  Implement baseline and constrained MVO using historical mean & covariance.  
- [6. Optimal Asset Allocation](#6-optimal-asset-allocation)  
  Derive efficient allocations across risk targets and compare concentration.  
- [7. Max Sharpe & Max Sortino Portfolios](#7-maximize-sharpe--sortino-portfolios)  
  Optimize explicitly for risk-adjusted return metrics and compare results.  

---

## Part III — Forward-Looking Analysis
- [8. Capital Market Expectations](#8-capital-market-expectations)  
  Introduce forward-looking assumptions and optional Black–Litterman.  
- [9. Forward-Looking Portfolio Statistics](#9-forward-looking-portfolio-statistics)  
  Recompute risk/return metrics; simulate outcomes (Monte Carlo, VaR, CVaR).  
- [10. Strategic Asset Allocation](#10-strategic-asset-allocation)  
  Define long-term policy portfolio; compare vs. optimized allocations.  

---

## Part IV — Backtesting & Evaluation
- [11. Backtesting](#11-backtesting)  
  Run rolling-window simulations with costs, turnover, and benchmarks:  
  - Equal Weight  
  - 60/40  
  - Inverse-Volatility  
  - Min-Variance  

---

## Conclusion & Next Steps
-  Summarize insights  
-  Highlight trade-offs (theory vs. implementation)  
-  Suggest future extensions (factor models, robust optimization, Black–Litterman)  


## 📊 Statistical Analysis of Portfolio and Individual Securities

### Introduction
This notebook begins the **Portfolio Optimization Project** by analyzing the statistical properties of both the **individual securities** and the **overall portfolio**. The purpose is the following:

1. **Understand the data**: Before applying optimization techniques, we need to explore the underlying behavior of the assets in terms of returns, risk, and correlations.  
2. **Establish baselines**: These descriptive statistics will serve as benchmarks against which optimization and constraints can be evaluated.

### Objectives
- Ingest and clean price data from Financial Modeling Prep
- Compute **daily log returns** and derive annualized metrics.  
- Perform statistical analysis at both the **asset level** and **portfolio level**:  
  - Expected return  
  - Volatility (standard deviation)  
  - Sharpe ratio (with constant risk-free rate)  
  - Skewness & kurtosis  
  - Correlation matrix
  - Sortino Ratio
  - Appraisal Ratio
- Visualize key results using plots and tables.  


## STEP #1: Portfolio and Individual ETFs data analysis

In [1]:
""" 
Import all necessary modules and get fmp key

"""



import pandas as pd
import numpy as np
import os
import sys
import dotenv
from dotenv import load_dotenv

import sys
# Go one level up to the project root
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from common.PortConnect import Port_Connect
from common.Portfolio import Portfolio_Stats

fmp_key = os.getenv("API_KEY")

port = Port_Connect(api_key=fmp_key)
portfolio = Portfolio_Stats()


#### Selecting appropiate benchmarks

SPY – SPDR S&P 500 ETF Trust - North American Large Cap Equities

EFA – iShares MSCI EAFE ETF - Developed Market ex-US Equities

EEM – iShares MSCI Emerging Markets ETF - Emerging Markets Equities

IEF – iShares 7-10 Year Treasury Bond ETF - US Treasuries (Fixed Income)

LQD – iShares iBoxx $ Investment Grade Corporate Bond ETF - US Investment Grade Corporate Bonds

VNQ – Vanguard Real Estate ETF - Real Estate (REITs)


In [2]:
spy = port.get_closing_prices('SPY',from_date='2005-01-01')
efa = port.get_closing_prices("EFA",from_date='2005-01-01')
eem = port.get_closing_prices("EEM",from_date='2005-01-01')
ief = port.get_closing_prices("IEF",from_date='2005-01-01')
lqd = port.get_closing_prices("LQD",from_date='2005-01-01')
vnq = port.get_closing_prices("VNQ",from_date='2005-01-01')

#### Analyzing each security independently

In [3]:
## Create a dataframe that holds all the securitis in one
benchmark = pd.concat([
    spy,efa,eem,ief,lqd,vnq
],axis=1)

benchmark



symbol,SPY,EFA,EEM,IEF,LQD,VNQ
date,,,,,,
2005-01-03,120.30,53.03,22.19,85.12,112.25,55.89
2005-01-04,118.73,52.02,21.51,84.59,111.62,55.05
2005-01-05,118.01,51.98,21.25,84.73,111.71,53.22
2005-01-06,118.61,51.98,21.23,84.81,111.79,53.63
2005-01-07,118.44,51.75,21.27,84.74,111.74,53.51
...,...,...,...,...,...,...
2025-08-27,646.63,91.68,49.91,96.01,110.03,91.98
2025-08-28,648.92,92.02,50.10,96.23,110.20,91.74
2025-08-29,645.05,91.48,49.86,96.15,109.80,92.24


In [ ]:
### Calculate returns from the specified ETFS
returns = benchmark.pct_change()

summary_list = []
names = []

### Calculate KPIS for portfolio analysis
for col in returns.columns:
    summary = portfolio.summary_stats(returns[col],market_index=returns["SPY"])
    summary_list.append(summary)
    names.append(col)
    
### Create summary of analysis for each ETF
pd.concat(summary_list,axis=0,keys=names)

,,Annualized Return,Annualized Vol,Annualized Semideviation,Sharpe Ratio,Skewness,Kurtosis,Cornish-Fisher VaR (5%),Historic CVaR (5%),Max Drawdown,Drawdown Duration (Days),Up Days %,Ulcer index,Calmar Ratio,Sortino Ratio,Beta,Correlation
SPY,0,0.084680,0.191921,0.156661,0.338931,0.005084,18.139516,0.015777,0.029536,-0.564737,1982,0.546259,14.171756,-0.149946,0.490799,1.000000,1.000000
EFA,0,0.026361,0.213237,0.169907,-0.020776,-0.049387,16.913542,0.018316,0.032608,-0.631823,6392,0.525486,26.026255,-0.041723,0.217492,0.975413,0.877907
EEM,0,0.040165,0.277675,0.208809,0.047272,0.536122,21.212108,0.019272,0.040447,-0.672349,4829,0.522985,26.269515,-0.059738,0.276151,1.179932,0.815535
IEF,0,0.005785,0.068784,0.045256,-0.519424,0.099950,5.678341,0.006736,0.009627,-0.277182,1856,0.509906,10.355715,-0.020872,0.116621,-0.105283,-0.293760
LQD,0,-0.001113,0.086586,0.071293,-0.423676,0.049027,60.616515,0.002543,0.012052,-0.293712,2366,0.519908,10.911132,0.003789,0.028948,0.086352,0.191401
VNQ,0,0.023774,0.289989,0.235021,-0.025720,0.020463,19.998179,0.023412,0.043539,-0.757649,2899,0.524909,25.220678,-0.031379,0.220574,1.128138,0.746624
